In [1]:
import time
import math
import collections
import numpy as np

In [2]:
f = open("data/entrain", "r")
g = open('data/endev',"r")
dev = g.read()
ex = f.read() + dev

emission_score = dict(collections.Counter(ex.split()))
#emission_score.pop('###/###')
input_seq = ex.split()
transition_score = {}
#input_seq.insert(0,'###/###')

for i in range(1,len(input_seq)):
    curr_tag = (input_seq[i-1].split('/'))[1]+'/'+(input_seq[i].split('/'))[1]
    if(curr_tag in transition_score):
        transition_score[curr_tag] = transition_score[curr_tag] + 1
    else:
        transition_score[curr_tag] = 1

#print(ex)
tag_dict = {}
word_dict = {}

for x in emission_score:
    if((x.split('/'))[1] in tag_dict):
        tag_dict[(x.split('/'))[1]] += emission_score[x]
    else:
        tag_dict[(x.split('/'))[1]] = emission_score[x]
for x in emission_score:
    if((x.split('/'))[0] in word_dict):
        word_dict[(x.split('/'))[0]] += emission_score[x]
    else:
        word_dict[(x.split('/'))[0]] = emission_score[x]
specific_vocab = {}
lambda1 = 0.01
for x in word_dict:
    for y in tag_dict:
        curr = x+'/'+y
        if(curr in emission_score):
            if(x in specific_vocab):
                specific_vocab[x].append(y)
            else:
                temp = []
                temp.append(y)
                specific_vocab[x] = temp
            emission_score[curr] += lambda1
        else:
            emission_score[curr] = lambda1
        tag_dict[y] += lambda1
        word_dict[x] += lambda1

#print(word_dict)
#print(tag_dict)
for x in emission_score:
    emission_score[x] = emission_score[x]/tag_dict[(x.split('/'))[1]]

tag_tag = {}
for x in transition_score:
    if((x.split('/'))[0] in tag_tag):
        tag_tag[(x.split('/'))[0]] += transition_score[x]
    else:
        tag_tag[(x.split('/'))[0]] = transition_score[x]
lambda2 = 0.99
for x in tag_tag:
    for y in tag_tag:
        curr = x+'/'+y
        if(curr in transition_score):
            transition_score[curr] += lambda2
        else:
            transition_score[curr] = lambda2
        tag_tag[y] += lambda2

for x in transition_score:
    transition_score[x] /= tag_tag[(x.split('/'))[0]]
    
vocabulary = []
for key in tag_tag.keys():
    k = key
    vocabulary.append(k)
#print(specific_vocab)
#print(vocabulary)
#print(emission_score)    
#print(tag_dict)
#print(tag_tag)
#print(transition_score)

In [5]:
start_time = time.time()
f = open("data/endev", "r")
ex2 = f.read()
ex = ex2.split()
#ex.insert(0,'###/###')
#ex.remove('###/###')

result = np.zeros((len(ex)))
for i in range(0,len(ex)):
    z = (ex[i].split('/'))[1]
    result[i] = vocabulary.index(z)


pi = np.zeros((len(vocabulary),len(ex)+1))
for i in range(0,len(vocabulary)):
    combo = (ex[0].split('/'))[0]+'/'+vocabulary[i]
    combo2 = '###/'+vocabulary[i]
    emit = 0
    if(combo in emission_score):
        emit = np.log(emission_score[combo])
    trans = np.log(transition_score[combo2])
    pi[i,0] = emit + trans
for i in range(1,len(ex)):
    for j in range(0,len(vocabulary)):
        combo = (ex[i].split('/'))[0]+'/'+vocabulary[j]
        emit = 1
        if(combo in emission_score):
            emit =  emission_score[combo]
        max_score = -float("inf")
        for k in range(0,len(vocabulary)):
            combo = vocabulary[k]+'/'+vocabulary[j]
            score = np.log(emit) + np.log(transition_score[combo]) + pi[k,i-1]
            max_score = max(score, max_score)
        pi[j,i] = max_score
            
x = np.argmax(pi,0)

"""
pi = np.zeros((len(vocabulary),len(ex)+1))
temp0 = (ex[0].split('/'))[0]
for i in range(0,len(vocabulary)):
    combo = temp0+'/'+vocabulary[i]
    combo2 = '###/'+vocabulary[i]
    emit = 0
    if(combo in emission_score):
        emit = np.log(emission_score[combo])
    trans = np.log(transition_score[combo2])
    pi[i,0] = emit + trans
for i in range(1,len(ex)):
    temp = (ex[i].split('/'))[0]
    for j in range(0,len(vocabulary)):
        combo = temp+'/'+vocabulary[j]
        emit = 1
        if(combo in emission_score):
            emit =  emission_score[combo]
        max_score = -float("inf")
        for k in range(0,len(vocabulary)):
            combo = vocabulary[k]+'/'+vocabulary[j]
            score = np.log(emit) + np.log(transition_score[combo]) + pi[k,i-1]
            max_score = max(score, max_score)
        pi[j,i] = max_score
"""
"""
if(temp0 in specific_vocab):
    vocab0 = specific_vocab[temp0]
    for i in range(0,len(vocab0)):
        combo = temp0+'/'+vocab0[i]
        combo2 = '###/'+vocab0[i]
        emit = 0
        if(combo in emission_score):
            emit = np.log(emission_score[combo])
        trans = np.log(transition_score[combo2])
        pi[i,0] = emit + trans
else:
    for i in range(0,len(vocabulary)):
        combo = temp0+'/'+vocabulary[i]
        combo2 = '###/'+vocabulary[i]
        emit = 0
        if(combo in emission_score):
            emit = np.log(emission_score[combo])
        trans = np.log(transition_score[combo2])
        pi[i,0] = emit + trans
for i in range(1,len(ex)):
    temp = (ex[i].split('/'))[0]
    if(temp0 in specific_vocab):
        vocab = specific_vocab[temp]
        for j in range(0,len(vocab)):
            combo = temp+'/'+vocab[j]
            emit = 1
            if(combo in emission_score):
                emit =  emission_score[combo]
            max_score = -float("inf")
            for k in range(0,len(vocab)):
                combo = vocab[k]+'/'+vocab[j]
                score = np.log(emit) + np.log(transition_score[combo]) + pi[k,i-1]
                max_score = max(score, max_score)
            pi[j,i] = max_score
    else:
        for j in range(0,len(vocabulary)):
            combo = temp+'/'+vocabulary[j]
            emit = 1
            if(combo in emission_score):
                emit =  emission_score[combo]
            max_score = -float("inf")
            for k in range(0,len(vocabulary)):
                combo = vocabulary[k]+'/'+vocabulary[j]
                score = np.log(emit) + np.log(transition_score[combo]) + pi[k,i-1]
                max_score = max(score, max_score)
            pi[j,i] = max_score

for i in range(0,len(vocabulary)):
    combo = (ex[-1].split('/'))[0]+'/'+vocabulary[i]
    combo2 = vocabulary[i]+'/###'
    emit = 0
    if(combo in emission_score):
        emit = np.log(emission_score[combo])
    trans = np.log(transition_score[combo2])
    pi[i,-1] += emit + trans
for i in range(len(ex)-2,-1,-1):
    for j in range(0,len(vocabulary)):
        combo = (ex[i].split('/'))[0]+'/'+vocabulary[j]
        emit = 1
        if(combo in emission_score):
            emit =  emission_score[combo]
        max_score = -float("inf")
        for k in range(0,len(vocabulary)):
            combo = vocabulary[j]+'/'+vocabulary[k]
            score = np.log(emit) + np.log(transition_score[combo]) + pi[k,i+1]
            max_score = max(score, max_score)
        pi[j,i] += max_score
"""
x = np.argmax(pi,0)
error = 0
for i in range(0,len(result)):
    if(result[i] - x[i]!=0.):
        error+=1
accuracy = (1 - (error/len(result)))*100
#print(pi)
print(accuracy)
#prediction = []
#print(len(result),len(ex))
out = ""
for i in range(0,len(result)):
    out += ex[i].split('/')[0]+'/'+vocabulary[x[i]]+'\n'
    #prediction.append(out)
    #print(ex[i], ex[i].split('/')[0]+'/'+vocabulary[x[i]])
    #print(ex[i], out)
#print(out)
#print(ex)
#print(prediction)
#for i in range(0,len(result)):
    #if(ex[i]==prediction[i]):
        #print("true")
    #else:
        #print("false")
p = open("output.txt","w")
p.write(out)
p.close()
print("--- %s seconds ---" % (time.time() - start_time))

    

SyntaxError: invalid syntax (<ipython-input-5-813f9889d1a9>, line 148)